In [1]:
import sys
sys.argv.append('--JOB_NAME')
sys.argv.append('James1')

sys.argv.append('--source_dataset')
sys.argv.append('s3://dataplatform-stg-landing-zone/housing/repairs-DLO/')

sys.argv.append('--source_address_column')
sys.argv.append('address_of_repair')

sys.argv.append('--source_postcode_column')
sys.argv.append('postcode_of_property')

sys.argv.append('--cleaned_addresses_s3_bucket_target')
sys.argv.append('s3://dataplatform-stg-refined-zone/housing/repairs-DLO/cleanedAddresses')

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
11,None,pyspark,idle,,,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
import sys
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job
from pyspark.ml import Pipeline
from pyspark.ml.feature import RegexTokenizer, NGram, HashingTF, MinHashLSH
from pyspark.sql.window import Window
from pyspark.sql.functions import rank, col, trim, when
import pyspark.sql.functions as F
from awsglue.dynamicframe import DynamicFrame

def get_glue_env_var(key, default="none"):
    if f'--{key}' in sys.argv:
        return getResolvedOptions(sys.argv, [key])[key]
    else:
        return default

## write into the log file with:
## @params: [JOB_NAME]
args = getResolvedOptions(sys.argv, ['JOB_NAME'])

source_dataset_path = get_glue_env_var('source_dataset', '')
source_address_column = get_glue_env_var('source_address_column', '')
source_postcode_column = get_glue_env_var('source_postcode_column', '')
cleaned_addresses_s3_bucket_target = get_glue_env_var('cleaned_addresses_s3_bucket_target', '')

sc = SparkContext.getOrCreate()
glueContext = GlueContext(sc)
logger = glueContext.get_logger()
job = Job(glueContext)
job.init(args['JOB_NAME'], args)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
logger.info('Fetch Source Data')
source_dataset = glueContext.create_dynamic_frame.from_catalog(
    name_space='dataplatform-stg-landing-zone-database',
    table_name='housing_repairs_dlo',
    transformation_ctx='source_dataset',
)

#(
#    connection_type="s3",
#    format="parquet",
#    connection_options={
#        "paths": [source_dataset_path],
#        "recurse": True,
#        'groupFiles': 'inPartition',
#    },
#    transformation_ctx="source_dataset"
#)

df = source_dataset.toDF()
source_dataset.printSchema()

An error was encountered:
Invalid status code '404' from http://localhost:8998/sessions/11 with error payload: {"msg":"Session '11' not found."}


In [8]:
def convertToNull(dfa):
   for i in dfa.columns:
       dfa = dfa.withColumn(i , when(col(i) == '', None).otherwise(col(i)))
   return dfa

def trimAllColumns(dfa):
   for c_name in dfa.columns:
       dfa = dfa.withColumn(c_name, trim(col(c_name)))
   return dfa

tmp = df[[
    'name_of_resident',
    'address_of_repair',
    'postcode_of_property',
    'import_date',
    'import_timestamp',
    'import_year'
]]
tmp = convertToNull(tmp)
tmp = trimAllColumns(tmp)

tmp = tmp.na.drop(subset=["name_of_resident"])
tmp = tmp.sort(col('name_of_resident'), col('import_date'))
#tmp = tmp.select(max('import_year'))

tmp.show()


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
"cannot resolve '`import_year`' given input columns: [make_a_note_if_the_resident_is_reporting_any_coronavirus_symptoms_in_the_household_and_advise_residents_to_wear_a_face_mask_when_the_operative_is_in_the_property_and_to_maintain_social_distancing, import_timestamp, postcode_of_property, uh_phone_number_1, block_reference, Timestamp, column29, job_description, uh_phone_number_2, if_yes__what_vulnerabilities_do_they_have?, is_there_a_cautionary_contact_alert_at_this_address?, form_reference_-_do_not_alter, if_there_is_a_cautionary_contact_alert__what_is_the_nature_of_it?, time_of_appointment, id, estate_reference, planners_notes, estate_name, is_the_job_a_recharge_or_sus_recharge?, planners_to_allocate_to_operatives, have_you_read_the_coronavirus_statement_to_the_resident?_please_advise_the_resident_to_wear_a_face_mask_when_the_operative_is_in_the_property_and_to_maintain_social_distancing, phone_number_of_resident, date_of_appointment, what_is_the_priority_f

In [ ]:
logger.info('adding new column')
df = df.withColumn('address', F.col(source_address_column))

logger.info('extract postcode into a new column')
df = df.withColumn('postcode', F.regexp_extract(F.col('address'), '([A-Za-z][A-Ha-hJ-Yj-y]?[0-9][A-Za-z0-9]? ?[0-9][A-Za-z]{2}|[Gg][Ii][Rr] ?0[Aa]{2})', 1))

logger.info('remove postcode from address')
df = df.withColumn('address', F.regexp_replace(F.col('address'), '([A-Za-z][A-Ha-hJ-Yj-y]?[0-9][A-Za-z0-9]? ?[0-9][A-Za-z]{2}|[Gg][Ii][Rr] ?0[Aa]{2})', ''))

logger.info('populate empty postcode with postcode from the other PC column')
df = df.withColumn("postcode", \
       F.when(F.col("postcode")=="" ,None) \
          .otherwise(F.col("postcode")))

if source_postcode_column:
    df = df.withColumn("postcode", F.coalesce(F.col('postcode'),F.col(source_postcode_column)))

logger.info('postcode formatting')
df = df.withColumn("postcode", F.upper(F.col("postcode")))
df = df.withColumn("postcode_nospace", F.regexp_replace(F.col("postcode"), " +", ""))
df = df.withColumn("postcode_length", F.length(F.col("postcode_nospace"))) 
df = df.withColumn("postcode_start", F.expr("substring(postcode_nospace, 1, postcode_length -3)"))
df = df.withColumn("postcode_end", F.expr("substring(postcode_nospace, -3, 3)"))
df = df.withColumn("postcode", F.concat_ws(" ", "postcode_start", "postcode_end"))

logger.info('address line formatting - remove commas and extra spaces')
df = df.withColumn("address", F.upper(F.col("address")))
df = df.withColumn("address", F.regexp_replace(F.col("address"), ",", ""))
df = df.withColumn("address", F.regexp_replace(F.col("address"), " +", " "))
df = df.withColumn("address", F.regexp_replace(F.col("address"), " ?- ?\z", ""))


logger.info('address line formatting - remove LONDON at the end (dont do this for out of London matching)')
df = df.withColumn("address", F.trim(F.col("address")))
df = df.withColumn("address_length", F.length(F.col("address"))) 
df = df.withColumn("address", \
       F.when(F.col("address").endswith(" LONDON"), F.expr("substring(address, 1, address_length -7)")) \
          .otherwise(F.col("address")))
                   
logger.info('address line formatting - remove HACKNEY at the end (dont necessarily this for out of borough matching)')
df = df.withColumn("address", F.trim(F.col("address")))
df = df.withColumn("address_length", F.length(F.col("address"))) 
df = df.withColumn("address", \
       F.when(F.col("address").endswith(" HACKNEY"), F.expr("substring(address, 1, address_length -8)")) \
          .otherwise(F.col("address")))

logger.info('address line formatting - dashes between numbers: remove extra spaces')
df = df.withColumn("address", F.regexp_replace(F.col("address"), '(\\d+) ?- ?(\\d+)', '$1-$2'))

logger.info('deal with abreviations')

logger.info('for \'street\': we only replace st if it is at the end of the string, if not there is a risk of confusion with saint')
df = df.withColumn("address", F.regexp_replace(F.col("address"), " ST.?\z", " STREET"))

df = df.withColumn("address", F.regexp_replace(F.col("address"), " RD.? ", " ROAD "))
df = df.withColumn("address", F.regexp_replace(F.col("address"), " RD.?\z", " ROAD"))

df = df.withColumn("address", F.regexp_replace(F.col("address"), " AVE ", " AVENUE "))
df = df.withColumn("address", F.regexp_replace(F.col("address"), " AVE\z", " AVENUE"))

df = df.withColumn("address", F.regexp_replace(F.col("address"), " HSE ", " HOUSE "))
df = df.withColumn("address", F.regexp_replace(F.col("address"), " HSE\z", " HOUSE"))

df = df.withColumn("address", F.regexp_replace(F.col("address"), " CT.? ", " COURT "))
df = df.withColumn("address", F.regexp_replace(F.col("address"), " CT.?\z", " COURT"))

# df = df.withColumn("address", F.regexp_replace(F.col("address"), " ST.? ", " SAINT "))

df = df.withColumnRenamed("address", "concatenated_string_to_match")

logger.info('create a unique ID')
df = df.withColumn("prinx", F.monotonically_increasing_id()).repartition(1)

logger.info('write into parquet')
cleanedDataframe = DynamicFrame.fromDF(df, glueContext, "cleanedDataframe")

parquetData = glueContext.write_dynamic_frame.from_options(
    frame=cleanedDataframe,
    connection_type="s3",
    format="parquet",
    connection_options={"path": cleaned_addresses_s3_bucket_target, "partitionKeys": []},
    transformation_ctx="parquetData")

job.commit()